In [ ]:
import matplotlib
matplotlib.use("module://matplotlib_inline.backend_inline")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Entorno Jupyter funcionando correctamente.")

In [ ]:
# === CARGA DE MICRODATOS EPH - Primer trimestre 2005 ===

# Ruta del archivo .dta (ajustada a tu PC)
ruta = r"C:\Users\julla\Downloads\Datos\Individual_t105.dta"

# Cargar base con pandas
df = pd.read_stata(ruta)

# Vista general
print("✅ Base cargada correctamente")
print("Filas y columnas:", df.shape)
df.head(5)


In [ ]:
# Información general
df.info()

# Variables disponibles
print("\nColumnas disponibles:\n")
print(df.columns.tolist())

# Vista rápida de los valores únicos en algunas columnas clave
cols_principales = ['CH04', 'CH06', 'CAT_OCUP', 'ESTADO', 'NIVEL_ED', 'P47T']
for col in cols_principales:
    if col in df.columns:
        print(f"\n{col}: {df[col].unique()[:10]}")


In [ ]:
# === CARGA DE MICRODATOS EPH - Primer trimestre 2025 ===

# Ruta del archivo .txt
ruta_2025 = r"C:\Users\julla\Downloads\Datos\usu_individual_T125.txt"

# Cargar base (delimitador punto y coma)
df_2025 = pd.read_csv(ruta_2025, sep=';', encoding='latin-1', low_memory=False)

# Vista general
print("✅ Base 2025 cargada correctamente desde TXT")
print("Filas y columnas:", df_2025.shape)
df_2025.head()


In [ ]:
print("Columnas 2005:", len(df.columns))
print("Columnas 2025:", len(df_2025.columns))

# Coincidencia de nombres
cols_2005 = set(df.columns)
cols_2025 = set(df_2025.columns)
coincidentes = cols_2005.intersection(cols_2025)
print("\nVariables coincidentes entre 2005 y 2025:", len(coincidentes))
print(sorted(list(coincidentes))[:20])


In [ ]:
# === Normalizar nombres de columnas ===
df_2025.columns = df_2025.columns.str.lower()  # pasar todo a minúsculas
df.columns = df.columns.str.lower()

# Verificar coincidencias nuevamente
cols_2005 = set(df.columns)
cols_2025 = set(df_2025.columns)
coincidentes = cols_2005.intersection(cols_2025)

print("🔁 Variables coincidentes entre 2005 y 2025:", len(coincidentes))
print(sorted(list(coincidentes))[:25])  # muestra las primeras 25


In [ ]:
# === AGREGAR VARIABLE DE AÑO Y UNIFICAR BASES ===

# Crear variable de año
df["anio"] = 2005
df_2025["anio"] = 2025

# Conservar solo columnas comunes
cols_comunes = list(coincidentes)

# Unir ambas bases (una sobre otra)
df_union = pd.concat([df[cols_comunes + ["anio"]], df_2025[cols_comunes + ["anio"]]], axis=0)

# Resumen
print("✅ Base combinada creada correctamente")
print("Filas totales:", df_union.shape[0])
print("Columnas totales:", df_union.shape[1])
print("\nAños disponibles:", df_union["anio"].unique())
df_union.head(3)


In [ ]:
!pip install missingno

In [ ]:
# === ANÁLISIS DE VALORES FALTANTES (Parte I.b) ===
import missingno as msno

# Calcular porcentaje de valores faltantes por variable
faltantes = df_union.isnull().mean() * 100
faltantes = faltantes.sort_values(ascending=False)

print("🔍 Porcentaje de valores faltantes por variable (top 15):")
print(faltantes.head(15))

# Visualización general de completitud
msno.matrix(df_union.sample(1000, random_state=1))  # muestra 1000 registros al azar
plt.title("🔢 Patrón de valores faltantes - muestra de 1000 observaciones")
plt.show()

# Heatmap de correlación de faltantes
msno.heatmap(df_union)
plt.title("🔥 Correlación de valores faltantes entre variables")
plt.show()


In [ ]:
# === PARTE I.c - Selección y limpieza de variables de interés ===

# 15 variables clave (según consigna y relevancia socioeconómica)
vars_interes = [
    'ch04',   # Sexo
    'ch06',   # Edad
    'ch07',   # Estado civil
    'ch08',   # Parentesco
    'nivel_ed',   # Nivel educativo
    'estado',     # Condición de actividad
    'cat_inac',   # Categoría de inactividad
    'cat_ocup',   # Categoría ocupacional
    'ipcf',       # Ingreso per cápita familiar
    'pp07g1',     # Ingreso laboral 1
    'pp07g2',     # Ingreso laboral 2
    'pp07h',      # Total de ingresos laborales
    'pp08d',      # Ingreso no laboral
    'pp08h',      # Total de ingresos del hogar
    'anio'        # Año de referencia
]

# Filtrar solo las variables disponibles
vars_presentes = [v for v in vars_interes if v in df_union.columns]
df_sel = df_union[vars_presentes].copy()

print(f"Variables seleccionadas ({len(vars_presentes)}): {vars_presentes}")
print("Filas iniciales:", df_sel.shape[0])

# Revisión de valores anómalos
for col in ['ipcf', 'pp07g1', 'pp07g2', 'pp07h', 'pp08d', 'pp08h']:
    if col in df_sel.columns:
        print(f"\n{col} - Valores únicos problemáticos:")
        print(df_sel[col].value_counts().head(10))

# Limpieza: eliminar negativos y codificaciones erróneas
for col in ['ipcf', 'pp07g1', 'pp07g2', 'pp07h', 'pp08d', 'pp08h']:
    if col in df_sel.columns:
        df_sel[col] = pd.to_numeric(df_sel[col], errors='coerce')  # asegurar tipo numérico
        df_sel.loc[df_sel[col] < 0, col] = np.nan  # eliminar negativos

# Confirmar limpieza
print("\n✅ Limpieza completada.")
print("Filas finales:", df_sel.shape[0])
df_sel.describe(include='all').T.head(10)


In [ ]:
# === Limpieza adicional de variables de ingreso ===

# Reemplazar "Sí"/"No" por NaN y asegurar tipo numérico
for col in ['pp07g1', 'pp07g2', 'pp07h']:
    if col in df_sel.columns:
        df_sel[col] = df_sel[col].replace({'Sí': np.nan, 'No': np.nan})
        df_sel[col] = pd.to_numeric(df_sel[col], errors='coerce')

# Recorte de valores extremos del IPCF (percentiles 1 y 99)
p1, p99 = df_sel['ipcf'].quantile([0.01, 0.99])
df_sel.loc[(df_sel['ipcf'] < p1) | (df_sel['ipcf'] > p99), 'ipcf'] = np.nan

print("✅ Limpieza final aplicada.")
print(df_sel.describe(include='all').T.loc[['ipcf','pp07g1','pp07g2','pp07h']])


In [ ]:
# === Ver aglomerados del año 2005 ===
df_union['region'].unique()
df_union['aglomerado'].unique()

In [ ]:
# === Ver aglomerados del año 2025 ===
df_union[df_union['anio'] == 2025]['aglomerado'].unique()

In [ ]:
# === Filtrar aglomerado seleccionado ===
aglomerado_sel = 'Partidos del GBA'
df_reg = df_union[df_union['aglomerado'] == aglomerado_sel].copy()

print(f"Aglomerado seleccionado: {aglomerado_sel}")
print("Filas:", df_reg.shape[0])
print("Años disponibles:", df_reg['anio'].unique())


In [ ]:
# Verificar distribución de códigos de aglomerado en 2025
df_union[df_union['anio'] == 2025]['aglomerado'].value_counts().head(10)


In [ ]:
# Verificar correspondencia entre aglomerado y región para 2025
df_2025 = df_union[df_union['anio'] == 2025][['aglomerado', 'region']].drop_duplicates()
print(df_2025.sort_values('aglomerado').head(20))


In [ ]:
# Verificar los nombres de región y aglomerado en 2025
# REGION (N)	Código de región macroeconómica	01 = Gran Buenos Aires ✅
# AGLOMERADO (N)	Código de aglomerado dentro de cada región	33 = Partidos del Gran Buenos Aires ✅

df_2025 = df_union[df_union['anio'] == 2025][['region', 'aglomerado']].drop_duplicates()
df_2025.sort_values('aglomerado')


In [ ]:
# === Selección coherente de Partidos del Gran Buenos Aires (2005 y 2025) ===

df_reg = df_union[
    (df_union['aglomerado'].isin(['Partidos del GBA', 33])) |
    (df_union['region'].isin([1]))  # seguridad extra si hay codificación numérica
].copy()

print("✅ Región/Aglomerado seleccionado: Partidos del Gran Buenos Aires (GBA)")
print("Filas totales:", df_reg.shape[0])
print("Años disponibles:", df_reg['anio'].unique())


In [ ]:
df_reg['nivel_ed'].value_counts(dropna=False)

In [ ]:
print(df_reg.columns.tolist())


In [ ]:
print(df_reg['nivel_ed'].unique()[:20])
print(df_reg['nivel_ed'].value_counts(dropna=False).head(10))

In [ ]:
print(df_reg['nivel_ed'].dropna().unique()[:20])
print(df_reg['nivel_ed'].dtype)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
sns.countplot(data=df_reg, x='ch04', hue='anio')
plt.title("Distribución por sexo - GBA 2005 vs 2025")
plt.xlabel("Sexo (ch04)")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
# --- Composición por sexo (%) - versión robusta ---
tmp = df_reg.copy()

# Normalizar ch04 a texto en todos los años
tmp["ch04"] = (
    tmp["ch04"]
    .replace({1: "Varón", 2: "Mujer", "1": "Varón", "2": "Mujer"})
    .fillna("Sin dato")
)

# Tabla de porcentajes por año
prop_df = (
    tmp.groupby(["anio", "ch04"])
       .size()
       .groupby(level=0, group_keys=False)
       .apply(lambda x: x / x.sum() * 100)
       .reset_index(name="Porcentaje")
)

# Forzar tipos/orden para asegurar colores correctos
prop_df["anio"] = prop_df["anio"].astype(str)  # "2005", "2025"
hue_order = ["2005", "2025"]
x_order   = ["Varón", "Mujer"]  # orden deseado en el eje X
palette   = {"2005": "#B2B2B2", "2025": "#4B4A67"}

# Gráfico
plt.figure(figsize=(7, 4))
sns.barplot(
    data=prop_df,
    x="ch04", y="Porcentaje", hue="anio",
    order=x_order, hue_order=hue_order,
    palette=palette, errorbar=None
)
plt.title("Composición por sexo (%) - GBA 2005 vs 2025", fontsize=12, weight="bold")
plt.xlabel("Sexo")
plt.ylabel("Porcentaje")
plt.legend(title="Año", loc="upper right")
plt.tight_layout()
plt.savefig("FIGURAS/composicion_sexo_porcentaje.png", dpi=300)
plt.show()


In [ ]:
# === INCISO 4 - MATRIZ DE CORRELACIÓN (EPH GBA 2005 vs 2025) ===

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

os.makedirs("FIGURAS", exist_ok=True)

# Variables de interés
vars_corr = ["ch04", "ch06", "ch07", "ch08", "nivel_ed", "estado", "cat_inac", "ipcf"]

# Subconjuntos
df_2005 = df_reg[df_reg["anio"] == 2005][vars_corr].copy()
df_2025 = df_reg[df_reg["anio"] == 2025][vars_corr].copy()

# Convertir variables numéricas
for col in ["ch06", "ipcf"]:
    df_2005[col] = pd.to_numeric(df_2005[col], errors="coerce")
    df_2025[col] = pd.to_numeric(df_2025[col], errors="coerce")

# Crear variables dummies sin drop_first para mostrar todas las categorías
df_2005_dummies = pd.get_dummies(df_2005, columns=["ch04","ch07","ch08","nivel_ed","estado","cat_inac"], drop_first=False)
df_2025_dummies = pd.get_dummies(df_2025, columns=["ch04","ch07","ch08","nivel_ed","estado","cat_inac"], drop_first=False)

# Calcular matrices de correlación
corr_2005 = df_2005_dummies.corr(numeric_only=True)
corr_2025 = df_2025_dummies.corr(numeric_only=True)

# --- Gráficos en disposición vertical ---
fig, axes = plt.subplots(2, 1, figsize=(12, 18))  # 2 filas, 1 columna

# Matriz 2005
sns.heatmap(corr_2005, ax=axes[0], cmap="coolwarm", center=0, cbar=False)
axes[0].set_title("Matriz de correlación - EPH 2005 (GBA)", fontsize=14, weight="bold")
axes[0].tick_params(axis='x', rotation=90, labelsize=8)
axes[0].tick_params(axis='y', labelsize=8)

# Matriz 2025
sns.heatmap(corr_2025, ax=axes[1], cmap="coolwarm", center=0, cbar=False)
axes[1].set_title("Matriz de correlación - EPH 2025 (GBA)", fontsize=14, weight="bold")
axes[1].tick_params(axis='x', rotation=90, labelsize=8)
axes[1].tick_params(axis='y', labelsize=8)

plt.tight_layout()
plt.savefig("FIGURAS/correlacion_2005_2025_vertical.png", dpi=300)
plt.show()


In [ ]:
# === INCISO 4 - MATRIZ DE CORRELACIÓN EPH GBA 2005 vs 2025 ===
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, os

os.makedirs("FIGURAS", exist_ok=True)

vars_corr = ["ch04","ch06","ch07","ch08","nivel_ed","estado","cat_inac","ipcf"]

# Filtrar por año
df_2005 = df_reg.loc[df_reg["anio"] == 2005, vars_corr].copy()
df_2025 = df_reg.loc[df_reg["anio"] == 2025, vars_corr].copy()

# --- Limpieza: reemplazar códigos inválidos o Ns/Nr ---
invalid_map = {
    "ch04": [9],
    "ch07": [0, 9],
    "ch08": [0, 9],
    "nivel_ed": [0, 9],
    "estado": [0, 9],
    "cat_inac": [0, 9]
}

for df in [df_2005, df_2025]:
    for col, invalid_vals in invalid_map.items():
        df[col] = df[col].replace(invalid_vals, np.nan)

# --- Convertir a numéricas las continuas ---
for col in ["ch06", "ipcf"]:
    df_2005[col] = pd.to_numeric(df_2005[col], errors="coerce")
    df_2025[col] = pd.to_numeric(df_2025[col], errors="coerce")

# --- Crear variables dummies legibles ---
map_prefix = {
    "ch04": "Sexo",
    "ch06": "Edad",
    "ch07": "EstadoCivil",
    "ch08": "CoberturaSalud",
    "nivel_ed": "Educacion",
    "estado": "Actividad",
    "cat_inac": "Inactividad",
    "ipcf": "IngresoPerCapita"
}

# Reorganizar: dummies solo para las categóricas
cats = ["ch04","ch07","ch08","nivel_ed","estado","cat_inac"]

df_2005_dum = pd.get_dummies(
    df_2005,
    columns=cats,
    prefix=[map_prefix[k] for k in cats],
    drop_first=False
).rename(columns={"ch06": "Edad", "ipcf": "IngresoPerCapita"})

df_2025_dum = pd.get_dummies(
    df_2025,
    columns=cats,
    prefix=[map_prefix[k] for k in cats],
    drop_first=False
).rename(columns={"ch06": "Edad", "ipcf": "IngresoPerCapita"})

# --- Calcular correlaciones ---
corr_2005 = df_2005_dum.corr(numeric_only=True)
corr_2025 = df_2025_dum.corr(numeric_only=True)

# --- Graficar heatmaps (verticales) ---
fig, axes = plt.subplots(2, 1, figsize=(13, 18))

sns.heatmap(corr_2005, ax=axes[0], cmap="coolwarm", center=0, cbar=False)
axes[0].set_title("Matriz de correlación - EPH 2005 (GBA)", fontsize=14, weight="bold")
axes[0].tick_params(axis='x', rotation=90, labelsize=8)
axes[0].tick_params(axis='y', labelsize=8)

sns.heatmap(corr_2025, ax=axes[1], cmap="coolwarm", center=0, cbar=False)
axes[1].set_title("Matriz de correlación - EPH 2025 (GBA)", fontsize=14, weight="bold")
axes[1].tick_params(axis='x', rotation=90, labelsize=8)
axes[1].tick_params(axis='y', labelsize=8)

plt.tight_layout()
plt.savefig("FIGURAS/correlacion_eph_2005_2025_final.png", dpi=300)
plt.show()


In [ ]:
# === INCISO 5 - Análisis de no respuesta en condición de actividad e ingreso ===
import os

# Asegurar que exista la carpeta de salida
os.makedirs("BASES", exist_ok=True)

# Contar no respuestas en condición de actividad
noresp_actividad = df_reg[df_reg["estado"].isin([0, 9])].groupby("anio").size()
print("Personas sin respuesta sobre condición de actividad:")
print(noresp_actividad)

# Crear subconjuntos según respuesta de ingreso total familiar (ITF)
respondieron = df_reg[df_reg["itf"] > 0].copy()
norespondieron = df_reg[(df_reg["itf"] == 0) | (df_reg["itf"].isna())].copy()

# Guardar dimensiones
print("\nDimensiones:")
print("Respondieron:", respondieron.shape)
print("No respondieron:", norespondieron.shape)

# Guardar bases intermedias (opcional)
respondieron.to_csv("BASES/respondieron.csv", index=False)
norespondieron.to_csv("BASES/norespondieron.csv", index=False)


In [ ]:
# === INCISO 5 - Análisis de no respuesta en condición de actividad e ingreso (2005 vs 2025) ===
import os
os.makedirs("BASES", exist_ok=True)

# --- 1. Personas sin respuesta sobre CONDICIÓN DE ACTIVIDAD ---
# Se consideran códigos 0, 9 o valores faltantes (NaN)
noresp_actividad = (
    df_reg[df_reg["estado"].isin([0, 9]) | df_reg["estado"].isna()]
    .groupby("anio")
    .size()
)
print("Personas sin respuesta sobre condición de actividad (por año):")
print(noresp_actividad)

# --- 2. Separar bases según respuesta de INGRESO TOTAL FAMILIAR ---
respondieron = df_reg[df_reg["itf"] > 0].copy()
norespondieron = df_reg[(df_reg["itf"] == 0) | (df_reg["itf"].isna())].copy()

# --- 3. Contar por año ---
resp_por_anio = respondieron.groupby("anio").size()
noresp_por_anio = norespondieron.groupby("anio").size()

print("\nCantidad de personas que RESPONDIERON ITF (por año):")
print(resp_por_anio)
print("\nCantidad de personas que NO RESPONDIERON ITF (por año):")
print(noresp_por_anio)

# --- 4. Guardar resultados ---
respondieron.to_csv("BASES/respondieron.csv", index=False)
norespondieron.to_csv("BASES/norespondieron.csv", index=False)

print("\nBases exportadas correctamente a la carpeta BASES/")


In [ ]:
# === INCISO 5 - Análisis de no respuesta: Condición de actividad e ingreso familiar ===

import pandas as pd, numpy as np, os

# Crear carpeta de salida
os.makedirs("BASES", exist_ok=True)

# --- 1. Detección de la variable correcta para condición de actividad ---
# Algunas bases 2005 usan 'cat_ocup' en lugar de 'estado'
if "estado" in df_reg.columns:
    var_act = "estado"
elif "cat_ocup" in df_reg.columns:
    var_act = "cat_ocup"
else:
    raise ValueError("No se encontró la variable de condición de actividad (estado o cat_ocup).")

# --- 2. Personas sin respuesta en condición de actividad ---
# Códigos de no respuesta (0, 9 o valores faltantes)
noresp_actividad = (
    df_reg[
        (df_reg[var_act].isin([0, 9])) | (df_reg[var_act].isna()) | (df_reg[var_act] == " ")
    ]
    .groupby("anio")
    .size()
)

print("Personas sin respuesta sobre condición de actividad (por año):")
print(noresp_actividad)

# --- 3. Separar bases según respuesta de ingreso total familiar ---
respondieron = df_reg[df_reg["itf"] > 0].copy()
norespondieron = df_reg[(df_reg["itf"] == 0) | (df_reg["itf"].isna())].copy()

# --- 4. Conteo resumido por año ---
resp_por_anio = respondieron.groupby("anio").size()
noresp_por_anio = norespondieron.groupby("anio").size()

print("\nCantidad de personas que RESPONDIERON ITF (por año):")
print(resp_por_anio)
print("\nCantidad de personas que NO RESPONDIERON ITF (por año):")
print(noresp_por_anio)

# --- 5. Guardar las bases intermedias ---
respondieron.to_csv("BASES/respondieron.csv", index=False)
norespondieron.to_csv("BASES/norespondieron.csv", index=False)

print("\nBases exportadas correctamente a la carpeta BASES/")


In [ ]:
import matplotlib.pyplot as plt

noresp_por_anio = pd.DataFrame({
    "Respondieron": resp_por_anio,
    "No respondieron": noresp_por_anio
})

noresp_por_anio.plot(
    kind="bar",
    stacked=True,
    color=["#4CAF50", "#F44336"],
    figsize=(7,4)
)

plt.title("Respuestas sobre ingreso total familiar (ITF) por año - GBA")
plt.xlabel("Año")
plt.ylabel("Cantidad de personas")
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Cargar el archivo con el path que encontraste
path_tabla = r"C:\Users\julla\Downloads\Datos\tabla_adulto_equiv.xlsx"

tabla_equiv = pd.read_excel(path_tabla)

# Mostrar información general
print("=== INFORMACIÓN GENERAL DE LA TABLA DE EQUIVALENCIAS ===")
print(tabla_equiv.info(), "\n")

# Mostrar las primeras filas
print("=== PRIMERAS FILAS DE LA TABLA ===")
print(tabla_equiv.head(10))

# Mostrar nombres de columnas exactos (para saber cómo mapearlos luego)
print("\n=== NOMBRES DE COLUMNAS ===")
print(list(tabla_equiv.columns))


In [ ]:
print(respondieron["ch04"].unique()[:20])
print(respondieron["ch04"].value_counts(dropna=False).head(10))

In [ ]:
# === CORREGIDO: Lectura robusta de tabla_adulto_equiv.xlsx ===
import pandas as pd, numpy as np

path_equiv = r"C:\Users\julla\Downloads\Datos\tabla_adulto_equiv.xlsx"

# Cargar todo sin encabezado
tabla_equiv_raw = pd.read_excel(path_equiv, header=None)

# Buscar fila donde empiece "Edad"
idx_inicio = tabla_equiv_raw[
    tabla_equiv_raw.iloc[:,0].astype(str).str.contains("Edad", case=False, na=False)
].index[0] + 1

# Extraer solo las tres columnas (Edad, Mujeres, Varones)
tabla_equiv = tabla_equiv_raw.iloc[idx_inicio:, [0,1,2]].copy()
tabla_equiv.columns = ["edad", "mujeres", "varones"]

# 🔹 LIMPIEZA ROBUSTA
tabla_equiv = tabla_equiv.dropna(subset=["edad"])             # eliminar filas vacías
tabla_equiv = tabla_equiv[~tabla_equiv["edad"].str.contains("Menor", na=False)]  # quitar textos
tabla_equiv = tabla_equiv[~tabla_equiv["edad"].str.contains("Edad", na=False)]   # quitar encabezados residuales

# Quitar la palabra "años" y convertir a numérico solo cuando sea posible
tabla_equiv["edad"] = (
    tabla_equiv["edad"]
    .astype(str)
    .str.replace(" años","", regex=False)
    .str.strip()
)

# Filtrar solo filas donde edad es numérica
tabla_equiv = tabla_equiv[tabla_equiv["edad"].str.match(r"^\d+$")]
tabla_equiv["edad"] = tabla_equiv["edad"].astype(int)

# Convertir valores de mujeres y varones a numéricos
tabla_equiv["mujeres"] = pd.to_numeric(tabla_equiv["mujeres"], errors="coerce")
tabla_equiv["varones"] = pd.to_numeric(tabla_equiv["varones"], errors="coerce")

# Mostrar verificación
print("✅ Tabla limpia y numérica:")
print(tabla_equiv.head(10))
print("\nColumnas:", tabla_equiv.columns.tolist())

In [ ]:
print(respondieron["ch04"].unique()[:20])
print(respondieron["ch04"].value_counts(dropna=False).head(10))

In [ ]:
# === REPARAR VARIABLE ch04 (sexo) PARA 2005 ===
import pandas as pd

# --- 1️⃣ Cargar base "respondieron" actual ---
respondieron = pd.read_csv("BASES/respondieron.csv", low_memory=False)
respondieron.columns = respondieron.columns.str.lower()

# --- 2️⃣ Cargar base original del 2005 (archivo .dta) ---
base_2005 = pd.read_stata(r"C:\Users\julla\Downloads\Datos\Individual_t105.dta")
base_2005.columns = base_2005.columns.str.lower()

print("Columnas en base 2005:", [c for c in base_2005.columns if "ch" in c or "hogar" in c or "codusu" in c][:12])

# --- 3️⃣ Filtrar solo variables necesarias para merge ---
cols_merge = [c for c in ["codusu", "nro_hogar", "componente", "ch04"] if c in base_2005.columns]
base_2005_sub = base_2005[cols_merge].copy()

# --- 4️⃣ Combinar para reinyectar el sexo en respondieron ---
respondieron = respondieron.merge(
    base_2005_sub,
    on=["codusu", "nro_hogar", "componente"],
    how="left",
    suffixes=("", "_from2005")
)

# Si ch04 está vacío en respondieron, usar el valor de la base 2005
respondieron["ch04"] = respondieron["ch04"].fillna(respondieron["ch04_from2005"])
respondieron.drop(columns=["ch04_from2005"], inplace=True)

# --- 5️⃣ Guardar resultado intermedio ---
respondieron.to_csv("BASES/respondieron_reparado.csv", index=False)
print("\n✅ Variable 'ch04' restaurada correctamente desde Individual_t105.dta")
print("Archivo generado: BASES/respondieron_reparado.csv")
print(respondieron[["anio", "ch04"]].head(10))

In [ ]:
# === INCISO 6 – Cálculo de adulto equivalente y suma por hogar ===
import pandas as pd, numpy as np, os

# --- 1️⃣ Crear tabla manual de equivalencias basada en el INDEC ---
tabla_equiv = pd.DataFrame({
    "edad_min": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 30, 46, 61, 76],
    "edad_max": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 29, 45, 60, 75, 120],
    "mujeres": [0.35, 0.37, 0.46, 0.51, 0.55, 0.60, 0.64, 0.66, 0.68, 0.69, 0.70, 0.72, 0.74, 0.76, 0.76, 0.77, 0.77, 0.77, 0.76, 0.77, 0.76, 0.67, 0.63],
    "varones": [0.35, 0.37, 0.46, 0.51, 0.55, 0.60, 0.64, 0.66, 0.68, 0.69, 0.79, 0.82, 0.85, 0.90, 0.96, 1.00, 1.03, 1.04, 1.02, 1.00, 1.00, 0.83, 0.74]
})

# Pasar a formato largo
tabla_equiv_long = tabla_equiv.melt(
    id_vars=["edad_min", "edad_max"],
    var_name="sexo_txt",
    value_name="adulto_equiv"
)

# Asignar códigos EPH (1 = varón, 2 = mujer)
tabla_equiv_long["sexo"] = tabla_equiv_long["sexo_txt"].map({"mujeres": 2, "varones": 1})
tabla_equiv_long = tabla_equiv_long[["edad_min", "edad_max", "sexo", "adulto_equiv"]]

print("Vista previa de tabla_equiv_long:")
print(tabla_equiv_long.head(10))

# --- 2️⃣ Cargar base respondieron ---
respondieron = pd.read_csv("BASES/respondieron_reparado.csv", low_memory=False)
respondieron.columns = respondieron.columns.str.lower()

# --- 🔧 Normalizar variable ch04 a formato numérico (1 = varón, 2 = mujer) ---
respondieron["ch04"] = (
    respondieron["ch04"]
    .replace({"Varón": 1, "Mujer": 2, "varon": 1, "mujer": 2})
    .astype(float)
)

# Convertir variables relevantes
respondieron["ch04"] = pd.to_numeric(respondieron["ch04"], errors="coerce")  # sexo
respondieron["ch06"] = pd.to_numeric(respondieron["ch06"], errors="coerce")  # edad

# --- 3️⃣ Asignar valor adulto_equiv según rango de edad y sexo ---
def buscar_equiv(edad, sexo):
    """Devuelve el adulto_equiv correspondiente al rango de edad y sexo."""
    if pd.isna(edad) or pd.isna(sexo):
        return np.nan
    fila = tabla_equiv_long[
        (tabla_equiv_long["sexo"] == int(sexo)) &
        (edad >= tabla_equiv_long["edad_min"]) &
        (edad <= tabla_equiv_long["edad_max"])
    ]
    return fila["adulto_equiv"].iloc[0] if not fila.empty else np.nan

respondieron["adulto_equiv"] = respondieron.apply(
    lambda row: buscar_equiv(row["ch06"], row["ch04"]),
    axis=1
)

# --- 4️⃣ Calcular total por hogar ---
respondieron["codusu"] = respondieron["codusu"].astype(str)
respondieron["nro_hogar"] = respondieron["nro_hogar"].astype(str)
respondieron["id_hogar"] = respondieron["codusu"] + "_" + respondieron["nro_hogar"]

adultos_equiv_hogar = (
    respondieron.groupby("id_hogar")["adulto_equiv"]
    .sum()
    .reset_index()
    .rename(columns={"adulto_equiv": "ad_equiv_hogar"})
)

respondieron = respondieron.merge(adultos_equiv_hogar, on="id_hogar", how="left")

# --- 5️⃣ Guardar resultado final ---
os.makedirs("BASES", exist_ok=True)
respondieron.to_csv("BASES/respondieron_equiv.csv", index=False)

print("\n✅ Archivo final generado: BASES/respondieron_equiv.csv")
print("Columnas agregadas: adulto_equiv, ad_equiv_hogar")

# --- 6️⃣ Verificar resultados por año ---
print(respondieron.groupby("anio")["adulto_equiv"].agg(["count", "mean", "min", "max"]))

In [ ]:
print(respondieron["adulto_equiv"].describe())
print("\nPorcentaje de valores faltantes en adulto_equiv:",
      respondieron["adulto_equiv"].isna().mean().round(3) * 100, "%")

print("\nEjemplo de hogares con más de 3 adultos equivalentes:")
print(respondieron[respondieron["ad_equiv_hogar"] > 3][["anio", "id_hogar", "ad_equiv_hogar"]].drop_duplicates().head(5))


In [ ]:
# === INCISO 7 – Ingreso necesario por hogar según la Canasta Básica Total ===
import pandas as pd, os

# Cargar base con equivalencias
respondieron = pd.read_csv("BASES/respondieron_equiv.csv", low_memory=False)

# Definir valores de la Canasta Básica Total por adulto equivalente
CBT = {2005: 205.07, 2025: 365177}

# Asignar la CBT correspondiente a cada año
respondieron["cbt_equiv"] = respondieron["anio"].map(CBT)

# Calcular ingreso necesario del hogar
respondieron["ingreso_necesario"] = respondieron["cbt_equiv"] * respondieron["ad_equiv_hogar"]

# Guardar nueva versión
os.makedirs("BASES", exist_ok=True)
respondieron.to_csv("BASES/respondieron_ingreso_necesario.csv", index=False)

# --- Resumen de validación ---
print("\n✅ Archivo actualizado: BASES/respondieron_ingreso_necesario.csv")
print("Columnas nuevas: cbt_equiv, ingreso_necesario\n")

# Mostrar algunas filas de ejemplo
print(respondieron[["anio", "id_hogar", "ad_equiv_hogar", "cbt_equiv", "ingreso_necesario"]].sample(10))

# Verificar rango de valores por año
print("\nResumen por año:")
print(respondieron.groupby("anio")["ingreso_necesario"].agg(["min", "mean", "max"]).round(2))


In [ ]:
# === ANÁLISIS DE ADULTOS EQUIVALENTES POR HOGAR (2005 vs 2025) ===
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1) Cargar base con ad_equiv_hogar e ingreso_necesario
df = pd.read_csv("BASES/respondieron_ingreso_necesario.csv", low_memory=False)
df.columns = df.columns.str.lower()

# Sanity checks mínimos
assert {"anio", "ad_equiv_hogar"}.issubset(df.columns), "Faltan columnas clave"
df = df.copy()
df["ad_equiv_hogar"] = pd.to_numeric(df["ad_equiv_hogar"], errors="coerce")
df["anio"] = pd.to_numeric(df["anio"], errors="coerce")

# 2) Resumen por año (no winsorizado)
resumen = (
    df.groupby("anio", observed=False)["ad_equiv_hogar"]
      .agg(n="count", mean="mean", median="median",
           p10=lambda s: s.quantile(0.10),
           p25=lambda s: s.quantile(0.25),
           p75=lambda s: s.quantile(0.75),
           p90=lambda s: s.quantile(0.90),
           max="max")
      .round(3)
)
print("=== Resumen por año (ad_equiv_hogar, sin winsorizar) ===")
print(resumen, "\n")

# 3) Umbrales de outliers (p99 por año) y winsorización
p99 = df.groupby("anio", observed=False)["ad_equiv_hogar"].quantile(0.99)
df = df.merge(p99.rename("p99"), on="anio", how="left")
df["ad_equiv_hogar_w"] = np.where(
    df["ad_equiv_hogar"] > df["p99"], df["p99"], df["ad_equiv_hogar"]
)

resumen_w = (
    df.groupby("anio", observed=False)["ad_equiv_hogar_w"]
      .agg(n="count", mean="mean", median="median",
           p10=lambda s: s.quantile(0.10),
           p25=lambda s: s.quantile(0.25),
           p75=lambda s: s.quantile(0.75),
           p90=lambda s: s.quantile(0.90),
           max="max")
      .round(3)
)
print("=== Resumen por año (winsorizado en p99 por año) ===")
print(resumen_w, "\n")

# 4) Bandas de tamaño equivalente
bins = [0, 2, 3, 4, 6, np.inf]
labels = ["≤2", "2–3", "3–4", "4–6", ">6"]
df["band_ad_eq"] = pd.cut(df["ad_equiv_hogar"], bins=bins, labels=labels, right=True)

dist_bandas = (
    df.groupby(["anio", "band_ad_eq"], observed=False)
      .size()
      .groupby(level=0).apply(lambda s: (100 * s / s.sum()).round(1))
      .unstack(fill_value=0)
)
print("=== Distribución % de hogares por bandas de ad_equiv_hogar ===")
print(dist_bandas, "\n")

# 5) Gráficos (comparables entre años)
sns.set(style="whitegrid")

# 5a) Densidades winsorizadas
plt.figure(figsize=(8, 4))
for a, sub in df.dropna(subset=["ad_equiv_hogar_w"]).groupby("anio", observed=False):
    sns.kdeplot(sub["ad_equiv_hogar_w"], label=str(a), linewidth=2)
plt.title("Densidad de adultos equivalentes por hogar (winsorizado p99)")
plt.xlabel("Adultos equivalentes del hogar")
plt.ylabel("Densidad")
plt.legend(title="Año")
plt.tight_layout()
os.makedirs("FIGURAS", exist_ok=True)
plt.savefig("FIGURAS/densidad_ad_equiv_hogar_p99.png", dpi=300)
plt.show()

# 5b) Boxplot comparativo
plt.figure(figsize=(7, 4))
sns.boxplot(data=df, x="anio", y="ad_equiv_hogar_w")
plt.title("Distribución de adultos equivalentes por hogar (winsorizado p99)")
plt.xlabel("Año")
plt.ylabel("Adultos equivalentes del hogar")
plt.tight_layout()
plt.savefig("FIGURAS/box_ad_equiv_hogar_p99.png", dpi=300)
plt.show()

# 5c) Barras apiladas por bandas (versión estable y retrocompatible)
dist_plot = dist_bandas.copy()
dist_plot = dist_plot.reset_index()  # convierte el índice 'anio' en columna
dist_plot = dist_plot.melt(id_vars="anio", var_name="banda", value_name="pct")

plt.figure(figsize=(7, 4))
sns.barplot(data=dist_plot, x="anio", y="pct", hue="banda")
plt.title("Estructura de hogares por adultos equivalentes (%)")
plt.xlabel("Año")
plt.ylabel("% de hogares")
plt.legend(title="Bandas ad_eq")
plt.tight_layout()
plt.savefig("FIGURAS/bandas_ad_equiv_hogar.png", dpi=300)
plt.show()


# 6) (Opcional) Chequeo identidad ingreso_necesario / cbt_equiv
if {"ingreso_necesario", "cbt_equiv"}.issubset(df.columns):
    df["check_cbt"] = (df["ingreso_necesario"] / df["cbt_equiv"]).round(3)
    gap = (df["check_cbt"] - df["ad_equiv_hogar"]).abs().median()
    print(f"Chequeo identidad ingreso_necesario/CBT = ad_equiv_hogar → mediana |gap| = {gap:.4f}")


In [ ]:
# === 8) Identificación de hogares pobres según ITF e ingreso_necesario ===

# Sanity check
assert {"itf", "ingreso_necesario", "anio"}.issubset(df.columns), "Faltan columnas clave"

# Convertir a numérico
df["itf"] = pd.to_numeric(df["itf"], errors="coerce")
df["ingreso_necesario"] = pd.to_numeric(df["ingreso_necesario"], errors="coerce")

# Crear variable binaria
df["pobre"] = np.where(df["itf"] < df["ingreso_necesario"], 1, 0)

# Resumen por año
res_pobreza = (
    df.groupby("anio", observed=False)
      .agg(
          hogares_pobres=("pobre", "sum"),
          total=("pobre", "count"),
          pct_pobres=("pobre", lambda s: round(100 * s.sum() / s.count(), 1))
      )
)
print("=== Pobreza por año (ITF < ingreso_necesario) ===")
print(res_pobreza, "\n")

# Visualización
plt.figure(figsize=(6,4))
sns.barplot(data=res_pobreza.reset_index(), x="anio", y="pct_pobres", color="steelblue")
plt.title("Porcentaje de hogares pobres según ITF vs ingreso necesario")
plt.xlabel("Año")
plt.ylabel("% de hogares pobres")
plt.tight_layout()
plt.savefig("FIGURAS/pobreza_por_anio.png", dpi=300)
plt.show()


In [ ]:
# Verificación adicional: razón ITF / ingreso_necesario
df["ratio_itf_nec"] = df["itf"] / df["ingreso_necesario"]

res_ratio = (
    df.groupby("anio", observed=False)["ratio_itf_nec"]
      .agg(media="mean", mediana="median", p25=lambda s: s.quantile(0.25), p75=lambda s: s.quantile(0.75))
      .round(2)
)
print("=== Razón ITF / ingreso_necesario por año ===")
print(res_ratio)


In [ ]:
# === 9) Estadísticas descriptivas relevantes de la variable 'pobre' ===

# Tabla descriptiva comparando 2005 vs 2025
desc_pobre = (
    df.groupby("anio", observed=False)
      .agg(
          pct_pobres=("pobre", lambda s: round(100 * s.mean(), 1)),
          media_itf=("itf", "mean"),
          mediana_itf=("itf", "median"),
          media_ad_eq=("ad_equiv_hogar", "mean"),
          mediana_ad_eq=("ad_equiv_hogar", "median")
      )
      .round(2)
)
print("=== Estadísticas descriptivas relevantes de 'pobre' por año ===")
print(desc_pobre, "\n")

# 1️⃣ Gráfico 1: Distribución de pobreza por tamaño del hogar
plt.figure(figsize=(7, 4))
sns.barplot(data=df, x="band_ad_eq", y="pobre", hue="anio", estimator=np.mean, errorbar=None)
plt.title("Proporción de hogares pobres por tamaño del hogar")
plt.xlabel("Adultos equivalentes (bandas)")
plt.ylabel("% de hogares pobres")
plt.legend(title="Año")
plt.gca().yaxis.set_major_formatter(lambda x, pos: f"{x*100:.0f}%")
plt.tight_layout()
plt.savefig("FIGURAS/pobreza_por_tamano_hogar.png", dpi=300)
plt.show()

# 2️⃣ Gráfico 2: Boxplot de ITF por condición de pobreza
plt.figure(figsize=(7, 4))
sns.boxplot(data=df, x="pobre", y="itf", hue="anio")
plt.title("Distribución del ingreso total familiar según condición de pobreza")
plt.xlabel("Condición de pobreza (0 = no pobre, 1 = pobre)")
plt.ylabel("Ingreso total familiar (ITF)")
plt.tight_layout()
plt.savefig("FIGURAS/box_itf_pobre.png", dpi=300)
plt.show()

# 3️⃣ (Opcional) Verificación numérica rápida de pobreza media por tamaño de hogar
check_band = (
    df.groupby(["anio", "band_ad_eq"], observed=False)["pobre"]
      .mean()
      .unstack()
      .round(2)
)
print("=== % de hogares pobres por tamaño del hogar ===")
print((check_band * 100).fillna(0))
